<a href="https://colab.research.google.com/github/MolecularFoundry/crucible-analysis-notebooks/blob/main/hyperspec-analysis/Hyperspectral_Explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Hyperspectral Confocal Microscopy Exploration and Plotting

This notebook is designed to analyze `hyperspec_picam_mcl` datasets from the `hip_microscope` confocal optical microscope in the Imaging Facility at the Molecular Foundry. The principles and code in this notebook can also be used to analyze other hyperspectral datasets, with some minor changes to loading of the `spec_map`, `h_array`,`v_arra`y, and `wls` data arrays.

## Initial Setup

In [ ]:
# For the default Google CoLab python enivronment,
# we need to install required packages that are not included by
# default. Note this step takes ~30seconds
# If this fails, remove the "> /dev/null" and try again to see the errors
!pip install jupyter_bokeh  > /dev/null

In [ ]:
# For Google CoLab, in order to access datasets store on Google Drive,
# we must mount the drives on the filesystem. This will ask for your
# permission to share your Google Drive with this notebook
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# To use interactive widgets in CoLab
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Required imports
import h5py
import numpy as np
import pandas as pd
import os
import glob
import matplotlib.pyplot as plt
import holoviews as hv
hv.extension('bokeh')


## Loading your Data File

In [ ]:
project_id = "MFP09238"

In [ ]:
dataset_options = glob.glob(f"/content/drive/Shareddrives/{project_id}/Datasets/*/*.h5")

print("Datasets found at the path above are: ")
for ds in dataset_options:
  print(ds)

In [ ]:
# Edit the location of the dataset you want to analyze (you can copy and paste from the output above)

file_path = ""

In [ ]:
# Load ScopeFoundry HDF5
with h5py.File(file_path, 'r') as f:
    M = f['measurement/hyperspec_picam_mcl']
    # 3D array of spectra (2D spatial dimensions)
    spec_map = np.array(M['spec_map'])[0]
    # positions of data points in um along x (horizontal) axis
    h_array = np.array(M['h_array'])
    # positions of data points in um along y (vertical) axis
    v_array = np.array(M['v_array'])
    # wavelengths of spectral values
    wls = np.array(M['wls'])
    # bounds of image
    imshow_extent = np.array(M['imshow_extent'])

## Plot Intensity Maps and Spectra


In [ ]:
img = spec_map.sum(axis=-1)
plt.figure()
vmin,vmax = np.percentile(img, [1,99])
plt.imshow(img, origin='lower',vmin=vmin,vmax=vmax)
plt.colorbar()

In [ ]:
avg_spec = spec_map.mean(axis=(0,1))
plt.figure()
plt.plot(wls, avg_spec)
plt.xlabel("wls")
plt.ylabel("intensity")

## Band Pass Images

In [ ]:
wl0 = 500 # Starting wavelength in nm
wl1 = 600 # Ending wavelength in nm

kk0,kk1 = np.searchsorted(wls, (wl0,wl1))
bandpass_image = spec_map[:,:,kk0:kk1].sum(axis=-1)
plt.figure()

vmin,vmax = np.percentile(bandpass_image, [1,99])
plt.imshow(bandpass_image, origin='lower',vmin=vmin,vmax=vmax, cmap=plt.cm.magma)
plt.colorbar()

## Interactive Data Exploration

In [ ]:
df = pd.DataFrame()

Ny, Nx, Nspec = spec_map.shape
X,Y = np.meshgrid(h_array,v_array)
II,JJ = np.meshgrid(np.arange(Nx), np.arange(Ny))
II.reshape(-1).shape
all_spectra = spec_map.reshape(-1,Nspec)
df['x']  = X.reshape(-1)
df['y']  = Y.reshape(-1)
df['ii'] = II.reshape(-1)
df['jj'] = JJ.reshape(-1)
df['total_intensity'] = spec_map.sum(axis=-1).reshape(-1)

df.head()


In [ ]:
hv.extension('bokeh')
from holoviews import opts
from holoviews.streams import Selection1D

df2 = df[['x','y']]

opts.defaults(opts.Points(tools=['box_select', 'lasso_select']))

point_map = hv.Points(df)

wlsmin = range(0,int(max(wls)-1))
wlsmax = range(1, int(max(wls)))
kdims = [hv.Dimension('wlsmin', label = 'Min. Wavelength', range = (int(min(wls)),int(max(wls)-1)), default=int(min(wls))),
         hv.Dimension('wlsmax', label = 'Max Wavelength', range = (int(min(wls)+1), int(max(wls))), default=int(max(wls)))]

index_stream = Selection1D(source=point_map)
print(index_stream)

def spec_map_bandpass_hvImage(wlsmin,wlsmax):
  kk0,kk1 = np.searchsorted(wls, [wlsmin, wlsmax])
  im = spec_map[:,:,kk0:kk1].mean(axis=-1)
  hvIm = hv.Image((h_array,v_array,im))
  hvIm.opts(aspect = "equal", cmap = "plasma")
  return hvIm


def selected_info(index):
    selected = point_map.iloc[index]
    #print(len(index), selected)
    #print(df.iloc[index])
    #specs = features[index] # N x Nspec
    #mean_spec = specs.mean(axis=0)
    #if index:
    #    label = 'Mean x, y: {}, {}'.format( tuple(selected.array().mean(axis=0)))
    #else:
    #label = 'No selection'
    #return selected.relabel(label).opts(color='red')
    return selected

selected_box = hv.DynamicMap(selected_info,
                              streams=[index_stream])

def select_spec(index):
  #print(index)
  if index == []:
    index = [0,]
  specs = all_spectra[index] # N x Nspec
  #print(specs.shape)
  mean_spec = specs.mean(axis=0)
  return hv.Curve(pd.DataFrame({'wls':wls,'intensity':mean_spec}))

spec_dmap = hv.DynamicMap(select_spec, streams=[index_stream])

dyn_specmap = hv.DynamicMap(spec_map_bandpass_hvImage, kdims=kdims)



lo = ( dyn_specmap * point_map)   + spec_dmap
display(lo)


In [ ]:
# Use selected spectra
selected_spec_indexes = index_stream.contents['index']
selected_specs = all_spectra[selected_spec_indexes]
print(f"Selected {len(selected_spec_indexes)} spectra")
spec_rows = df.iloc[selected_spec_indexes]

plt.figure()
plt.imshow(spec_map.sum(axis=-1),origin='lower',extent=imshow_extent)
plt.scatter(spec_rows['x'], spec_rows['y'], color='w', marker='x')
plt.gca().set_aspect(1)
plt.figure()
for i in selected_spec_indexes:
  row = df.iloc[i]
  plt.plot(all_spectra[i], label=f"{i} [{row['x']}, {row['y']}]")
plt.plot(selected_specs.mean(axis=0), label='mean spectrum')

if len(selected_spec_indexes) < 15:
  plt.legend()